In [4]:
import os
import csv
import struct
import numpy as np
from time import time
from numpy import random as rnd
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from IPython.display import clear_output
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
!ls

Logistic Regression.ipynb NHIS_OPEN_GJ_2017_100.csv NHIS_OPEN_GJ_2018_100.csv


In [ ]:
"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""
def read(dataset = "training", path = "./", year = None):
    
    year_of_dataset = 'NHIS_OPEN_GJ_2017_100.csv' if year == 2017 else 'NHIS_OPEN_GJ_2018_100.csv'

    if dataset is "training":
        fname_img = os.path.join(path, year_of_datset)
#         fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
        print('read() training!!')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')
        print('read() testing!!')
    else:
        raise Exception("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

read()
# pyplot.show()

In [56]:
import csv
from typing import *
def read(year:int = 2017, no_npz = True):
    f = open('NHIS_OPEN_GJ_2017_100.csv', 'r', encoding='CP949')
    rdr = csv.reader(f)
    infos = ['기준년도', '가입자일련번호', '성별코드', 
     '연령대코드(5세단위)', '시도코드', '신장(5Cm단위)', 
     '체중(5Kg단위)', '허리둘레', '시력(좌)', '시력(우)', 
     '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', 
     '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드', 
     'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백', 
     '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', 
     '감마지티피', '흡연상태', '음주여부', '구강검진수검여부', 
     '치아우식증유무', '결손치유무', '치아마모증유무', 
     '제3대구치(사랑니)이상', '치석', '데이터공개일자']
    data = []
    labels = []
    # 공복혈당 = 14
    FBS_INDEX = 14
    DATASIZE = 1000
    required_info_index = [2, 3, 5, 6, 7, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
    data_infos = []

    for n, col in enumerate(rdr):
        if n == 0:
            data_infos.append([info for index, info in enumerate(col) if index in required_info_index])
        else:
            data.append([info for index, info in enumerate(col) if index in required_info_index])
            labels.append(1) if int(col[FBS_INDEX]) >= 120 else labels.append(0)
        if n >= DATASIZE:
            break

    # print(data_infos)
    # for d in data: print(d)
    # for l in labels: print(l)
    # print(data)
    f.close()
    X_train = np.array(data)
    y_train = np.array(labels)
    print(len(X_train), len(y_train))


In [57]:
read()

1000 1000


In [46]:
a = [1,3,5]
b = [['2017', '15', '2', '7', '41', '160', '50', '76', '0.9', '1', '1', '1', '129', '77', '79', '205', '219', '53', '108', '14.5', '1', '0.6', '21', '24', '58', 
      '1', '0', '0', '', '', '', '', '', '20181126'],
     ['2017', '16', '1', '9', '47', '170', '65', '80', '1', '1', '1', '1', '113', '72', '104', '113', '35', '44', '62', '16', '2', '0.9', '15', '10', '12', '1',
      '0', '0', '', '', '', '', '', '20181126'],
     ['2017', '17', '2', '6', '42', '160', '65', '73', '1.2', '0.9', '1', '1', '126', '78', '96', '148', '60', '54', '82', '12.3', '1', '0.5', '12', '7', '9', 
      '1', '1', '0', '', '', '', '', '', '20181126']]
c = []
for x in b:
    c.append([y for i, y in enumerate(x) if i in a])
print(c)
X_train = np.array(b)
print(X_train)

[['15', '7', '160'], ['16', '9', '170'], ['17', '6', '160']]
[['2017' '15' '2' '7' '41' '160' '50' '76' '0.9' '1' '1' '1' '129' '77'
  '79' '205' '219' '53' '108' '14.5' '1' '0.6' '21' '24' '58' '1' '0' '0'
  '' '' '' '' '' '20181126']
 ['2017' '16' '1' '9' '47' '170' '65' '80' '1' '1' '1' '1' '113' '72'
  '104' '113' '35' '44' '62' '16' '2' '0.9' '15' '10' '12' '1' '0' '0' ''
  '' '' '' '' '20181126']
 ['2017' '17' '2' '6' '42' '160' '65' '73' '1.2' '0.9' '1' '1' '126' '78'
  '96' '148' '60' '54' '82' '12.3' '1' '0.5' '12' '7' '9' '1' '1' '0' ''
  '' '' '' '' '20181126']]


In [5]:
# read dataset fron file.

#tr = list(read("training", "/content/gdrive/My Drive/"))
#ts = list(read("testing","/content/gdrive/My Drive/"))
data_2017 = list(read("training", "./NHIS_OPEN_GJ_2017_100.csv"))
print(len(data_2017))

# LIMIT = 5000
# X_train = np.array(list(zip(*tr))[1][:LIMIT])
# y_train = np.array(list(zip(*tr))[0][:LIMIT])

# X_test = np.array(list(zip(*ts))[1][:LIMIT])
# y_test = np.array(list(zip(*ts))[0][:LIMIT])

#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)

NameError: name 'read' is not defined